In [1]:
import sys
sys.path.insert(0, '../tree/')
import pandas as pd
import numpy as np
from simulate_tree_data import simulate_tree_data
import copy
import time

In [2]:
ts = int(time.time())

In [7]:
node_list = [[[13, 2, 2], 8, [4, 4], [2, 22]], 
             [2, 13], [[2, 2, 2, 2, 2], 10], [[2, 2, 4], [2, 6]], 
             [4, 4, [8, 2]]]
alpha_true = [0.4, -0.3, 0.3]
base_rate_true = 0.1
gamma_list = [0.4, 0.2, 0.1, .05]
n_cov = 3
meas_std = .01
data, X = simulate_tree_data(base_rate_true, alpha_true,meas_std, node_list, gamma_list,
                                 add_cov=True, center_u=False, add_noise=False, cov_noise=True, seed=ts)
df = pd.DataFrame(data)

In [8]:
np.linalg.cond(X)

8.6467856174080744

In [9]:
cov_eff = 0.
for i,row in df.iterrows():
    val = 0.0
    for j in range(n_cov):
        val += df.loc[i,'cov'+str(j+1)]*alpha_true[j]
    cov_eff += abs(val)
print('mean covariate effects',cov_eff/df.shape[0])

mean covariate effects 0.366760986298


In [10]:
re = []
for i,row in df.iterrows():
    u = 0.0
    for j in range(3):
        if not pd.isna(row['level_'+str(j)+'_u']):
            u += row['level_'+str(j)+'_u']
    re.append(abs(u))
print('mean random effects magnitude',np.mean(np.array(re)))

mean random effects magnitude 0.604815415513


In [12]:
file_name = '../data/test/sim_data_tiny_' + str(ts) +'.csv'
df.to_csv(file_name)